In [ ]:
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import numpy as np
import glob

In [ ]:
# define model path
model_path = 'best (1).pt'
model = YOLO(model_path)

In [ ]:

# Define your input, output, and mask directories
input_dir = r'E:\car images\AWSImages2\test\pics'
output_dir = r'E:\car images\AWSImages2\test\finalds'
mask_dir = r'E:\car images\AWSImages2\test\binaryimage'

image_files = glob.glob(os.path.join(input_dir, '*'))

for image_file in image_files:
    # Read the image
    original_image = cv2.imread(image_file)
    
    # Predict the mask using your model
    results = model(original_image)
    
    # Check if results are None
    if results is None:
        print(f"Skipping image {image_file} - Model prediction returned None.")
        continue
    
    # Extract mask from results
    if len(results) > 0 and results[0].masks is not None and len(results[0].masks.data) > 0:
        mask = results[0].masks.data[0].cpu().numpy() * 255  # Modify based on your model's output
    else:
        print(f"Skipping image {image_file} - No valid mask found in results.")
        continue
    
    # Resize the mask to match the original image size
    mask = cv2.resize(mask, (original_image.shape[1], original_image.shape[0]))

    # Convert the mask to a binary mask (0 or 255 values)
    _, binary_mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    # Calculate the area of the mask
    area = np.sum(binary_mask > 0)

    # Check if the area is greater than 30% of total pixels
    total_pixels = original_image.shape[0] * original_image.shape[1]
    if area <= total_pixels * 0.3:
        print(f"No car detected in image {image_file} or the area is not more than 30% of the total pixels. Skipping...")
        continue

    # Make sure the data types are compatible
    original_image = original_image.astype(np.uint8)
    binary_mask = binary_mask.astype(np.uint8)

    # Use the bitwise_and operation to apply the mask to the original image
    result = cv2.bitwise_and(original_image, original_image, mask=binary_mask)

    # Save the result in the output directory with the same filename
    output_file = os.path.join(output_dir, os.path.basename(image_file))
    cv2.imwrite(output_file, result)

    # Save the mask in the mask directory with the same filename
    mask_file = os.path.join(mask_dir, os.path.basename(image_file))
    cv2.imwrite(mask_file, binary_mask)

    print(f"Processed and saved image: {image_file}")

print("Processing complete.")